In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-OPUS-vi-en"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)

/home/chinh/clc_data/Script/Faiss-EM-OPUS-vi-en20210302-2208


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
!mkdir data_bin
!unzip 'opus-100-corpus.zip' -d 'data_bin'

--2021-03-02 22:08:49--  https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
Resolving github.com (github.com)... 13.229.188.59
Connecting to github.com (github.com)|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip [following]
--2021-03-02 22:08:49--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30690851 (29M) [application/zip]
Saving to: ‘opus-100-corpus.zip’

opus-100-corpus.zip 100%[===================>]  29,27M  1,17MB/s    in 27s     

2021-03-02 22:09:19 (1,07 MB/s) - ‘opus-100-corpus.zip’ saved [30690851/30690851]

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/opus.en-vi-dev.en', 'data_bin/opus.en-vi-test.en', 'data_bin/opus.en-vi-train.en']
['data_bin/opus.en-vi-dev.vi', 'data_bin/opus.en-vi-test.vi', 'data_bin/opus.en-vi-train.vi']
1004000
1004000


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

803200
803200
100400
100400
100400
100400
Childs?

Childs?

It's powered up now, isn't it?

Bây giờ nó bật rồi phải không nhỉ?

Why the hell are you going to Cuddy's baby naming?

Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 105172
drwxrwxr-x 3 chinh chinh     4096 Thg 3   2 22:09 .
drwxrwxr-x 6 chinh chinh     4096 Thg 3   2 22:08 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 3   2 22:09 data_bin
-rw-rw-r-- 1 chinh chinh  3318348 Thg 3   2 22:09 en_test
-rw-rw-r-- 1 chinh chinh 26563373 Thg 3   2 22:09 en_train
-rw-rw-r-- 1 chinh chinh  3328557 Thg 3   2 22:09 en_valid
-rw-rw-r-- 1 chinh chinh 30690851 Thg 3   2 22:09 opus-100-corpus.zip
-rw-rw-r-- 1 chinh chinh  4365721 Thg 3   2 22:09 vi_test
-rw-rw-r-- 1 chinh chinh 35019159 Thg 3   2 22:09 vi_train
-rw-rw-r-- 1 chinh chinh  4382771 Thg 3   2 22:09 vi_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../../../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  2.1136631965637207


In [8]:
srcfile = "vi_train"
tarfile = "en_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = vi_train
lst_tarfile = en_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)

Test Cosin similarity

In [9]:
import faiss

start = time.time()
vec_lst_srcfile = model.embed_sentences(lst_srcfile)
index = faiss.IndexFlatL2(600)
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  2387.2361369132996
803200
803200
[[ 988 2012 2495 2792 2743    0]
 [  21   25   66   70    6    1]
 [   2    3    4    8    7    5]
 ...
 [   2    3    4    8    7    5]
 [ 293  774  922  940  917  844]
 [   2    3    4    8    7    5]]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [10]:
print(type(vec_lst_srcfile))
print(vec_lst_srcfile.shape)


<class 'numpy.ndarray'>
(803200, 600)


In [11]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

1
0.0
0.0


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.9, 0.8, 0.7, 0.6]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("vi_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("vi_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("vi_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.9 EM sentences = 2357 / 803200 0.002934511952191235 %
threshold = 0.8 EM sentences = 2985 / 803200 0.0037163844621513945 %
threshold = 0.7 EM sentences = 3685 / 803200 0.004587898406374502 %
threshold = 0.6 EM sentences = 4446 / 803200 0.005535358565737052 %
=== Ending, Total time (second):  26.269177675247192
